# Fitting the Brown Sample with Prospector and Dynesty

This is an interactive version of the demo demonstrating `dynesty` usage on the Brown et al. (2014) sample of galaxies.

First let's set up some environmental dependencies. These just make the numerics easier and adjust some of the plotting defaults to make things more legible.

In [ ]:
import time, sys, os
import h5py
import numpy as np
from matplotlib.pyplot import *

%matplotlib inline

In [ ]:
# re-defining plotting defaults
from matplotlib.font_manager import FontProperties
from matplotlib import gridspec
rcParams.update({'xtick.major.pad': '7.0'})
rcParams.update({'xtick.major.size': '7.5'})
rcParams.update({'xtick.major.width': '1.5'})
rcParams.update({'xtick.minor.pad': '7.0'})
rcParams.update({'xtick.minor.size': '3.5'})
rcParams.update({'xtick.minor.width': '1.0'})
rcParams.update({'ytick.major.pad': '7.0'})
rcParams.update({'ytick.major.size': '7.5'})
rcParams.update({'ytick.major.width': '1.5'})
rcParams.update({'ytick.minor.pad': '7.0'})
rcParams.update({'ytick.minor.size': '3.5'})
rcParams.update({'ytick.minor.width': '1.0'})
rcParams.update({'xtick.color': 'k'})
rcParams.update({'ytick.color': 'k'})
rcParams.update({'font.size': 30})

**Prospector** utilizes three main packages:
- **fsps**, which governs the fundamental stellar population synthesis models (via the **python-fsps** package),
- **sedpy**, which contains some routines for computing projecting spectra onto filter bandpasses, and
- **prospect**, which is where the likelihood evaluations, parameter priors, and posterior sampling takes place.

Let's import those now.

In [ ]:
import fsps
import sedpy
import prospect

## Setup

We now want to initialize our model.

In [ ]:
from prospect.models import model_setup

The overall or meta-parameters controlling how the fit is done are stored in the ``run_params`` dictionary, which is a mix of options
  1. defined in the parameter file
  2. specified at the command line (if not running in interactive mode.)

Let's load them from the parameter file and look at them:

In [ ]:
clargs = {'param_file': 'brownseds_agn_params.py'}
run_params = model_setup.get_run_params(argv='brownseds_agn_params.py', **clargs)
print(run_params)

Our model will be composed of four components:
- an **stellar population synthesis (SPS)** model for the underlying physical parameters,
- an underlying **statistical model** composed of a set of parameters, priors, etc., 
- a **noise model** for the underlying calibration vector, and
- a set of **observations** we are trying to fit.

See the input parameter file for additional info on the specific parameters we are initializing here.

The SPS model specified in the ``load_obs`` method of the parameter file can be one of several different types, corresponding to different SFH parameterizations.

In [ ]:
# load sps model
sps = model_setup.load_sps(**run_params)
print(sps)

For now we set the noise models to ``None``, which means that the default simple $\chi^2$ style likelihood will be used.

In [ ]:
# load noise model (none)
spec_noise, phot_noise = model_setup.load_gp(**run_params)

The model object keeps track of the fit parameters and their priors.

In [ ]:
# load parameter model
model = model_setup.load_model(**run_params)

The `obs` dictionary contains all our observational data, from which we are attempting to infer posterior PDFs for the parameters.  Guidelines for units and what keys must be in the `obs` dictionary can be found in the documentation.

In [ ]:
# demo data (generated from the script)
obs = model_setup.load_obs(**run_params)
print(run_params['objname'])

Now that we've initialized the components of our model, we need to define how we want to compare our model to the data by establishing the appropriate **likelihood**. In most cases, this will simply be a function of the **spectral likelihood** and a **photometric likelihood** such that

$$
\mathcal{L} = f(\mathcal{L}_{\textrm{spec}}, \mathcal{L}_{\textrm{phot}}) \quad .
$$

Assuming our errors are Normal (i.e. Gaussian), the log-likelihoods for each component are extremely straightforward to define and can be imported directly from Prospector.

In [ ]:
from prospect.likelihood import lnlike_spec, lnlike_phot, write_log

How we choose to combine these likelihoods might vary depending on the particulars of our data. For the demo, our likelihood function for our model parameters $\boldsymbol{\theta}$ is just

$$
\ln\mathcal{L}(\boldsymbol{\theta}) = \ln\mathcal{L}_{\textrm{spec}}(\boldsymbol{\theta}) + \ln\mathcal{L}_{\textrm{phot}}(\boldsymbol{\theta}) \quad .
$$

In [ ]:
def lnprobfn(theta):
    """Given a parameter vector, a dictionary of observational data 
    and a model object, return the ln of the posterior. 
    This requires that an sps object (and if using spectra 
    and gaussian processes, a GP object) be instantiated.
    """

    # Calculate prior probability and exit if not within prior
    lnp_prior = model.prior_product(theta, nested=True)
    if not np.isfinite(lnp_prior):
        return -np.infty
        
    # Generate mean model
    try:
        spec, phot, x = model.mean_model(theta, obs, sps=sps)
    except(ValueError):
        return -np.infty
    vectors = {}  # This would be used for noise model weight functions

    # Calculate likelihoods
    lnp_spec = lnlike_spec(spec, obs=obs, spec_noise=spec_noise)
    lnp_phot = lnlike_phot(phot, obs=obs, phot_noise=phot_noise)

    return lnp_prior + lnp_phot + lnp_spec

## Running Prospector

### Model Preview

Let's take a quick look at our model/data to get a sense of what we're dealing with and what we're fitting for.

In [ ]:
print 'Free params:', model.free_params
print 'Fixed params:', model.fixed_params

Since `z_fraction` is a vector that includes 5 parameters (as the non-parametric SFH), this gives us a total of 15 parameters.

### SED Preview

Let's now see what our model and data look like.

In [ ]:
wspec = sps.wavelengths  # *restframe* spectral wavelengths
a = 1.0 + model.params.get('zred', 0.0)  # cosmological redshifting

# photometric effective wavelengths
wphot = np.array([f.wave_effective for f in obs['filters']])

# initial parameters
initial_theta = model.rectify_theta(model.initial_theta)

# generate model
out_init = model.mean_model(initial_theta, obs, sps=sps) 
mspec_init, mphot_init, mextra_init = out_init

# establish bounds
xmin, xmax = np.min(wphot) * 0.8, np.max(wphot) / 0.8
temp = np.interp(np.linspace(xmin, xmax, 10000), wspec * a, mspec_init)
ymin, ymax = temp.min() * 0.8, temp.max() / 0.8

# set up figure
figure(figsize=(16, 8))

# plot model + data
loglog(wspec * a, mspec_init, label='Model spectrum', 
       lw=0.7, color='navy', alpha=0.7)
errorbar(wphot, mphot_init, label='Model photometry', 
         marker='s', markersize=10, alpha=0.8, ls='', lw=3,
         markerfacecolor='none', markeredgecolor='blue', 
         markeredgewidth=3)
errorbar(wphot, obs['maggies'], yerr=obs['maggies_unc'], 
         label='Observed photometry',
         marker='o', markersize=10, alpha=0.8, ls='', lw=3,
         ecolor='red', markerfacecolor='none', markeredgecolor='red', 
         markeredgewidth=3)

# plot filters
for f in obs['filters']:
    w, t = f.wavelength.copy(), f.transmission.copy()
    while t.max() > 1:
        t /= 10.
    t = 0.1 * (ymax - ymin) * t + ymin
    loglog(w, t, lw=3, color='gray', alpha=0.7)

# prettify
xlabel('Wavelength [A]')
ylabel('Flux Density [maggies]')
xlim([xmin, xmax])
ylim([ymin, ymax])
legend(loc='best', fontsize=20)
tight_layout()

### Sampling the Posterior

Now that we've got all the objects set up, we can begin sampling from the posterior using **Nested Sampling**. Prospector by default uses **emcee**, but here we will use [**dynesty**](https://github.com/joshspeagle/dynesty). For nested sampling we need to define a prior transfrom function. This is easy with the prior objects in Prospector, where a prior_transform method is defined by the model object. We also need to make sure that the prior probability calculation in the posterior probability function is appropriate for nested sampling (note the `nested=True` option in `model.prior_product` in the `lnprobfn` above).

In [ ]:
def prior_transform(u):        
    """Simple wrapper for the model prior transform."""
    
    return model.prior_transform(u)

Note that creating a new model with FSPS is somewhat time-intensive, but once the relevant model(s) have been loaded they are subsequently stored in cache so similar models can be generated much more quickly.

Now lets import **dynesty**.

In [ ]:
import dynesty

### Sampling in Parallel

dynesty supports parallel operations through the use of a user-provided pool. Let's set up a pool now.

In [ ]:
# import ipyparallel
import ipyparallel as ipp
rc = ipp.Client()  # create a client (running MPI)
dview = rc[:]  # create a `DirectView` object
dview.use_dill();  # use `dill` for advanced pickling
nprocs = len(rc.ids)  # number of MPI processes available

print(rc.ids)

In [ ]:
%%px

# import environment
import time, sys, os
import h5py
import numpy as np
import fsps
import sedpy
import prospect
from prospect.models import model_setup
from prospect.likelihood import lnlike_spec, lnlike_phot, write_log
from prospect.io import write_results
import dynesty

# set up model
clargs = {'param_file': 'brownseds_agn_params.py'}
run_params = model_setup.get_run_params(argv='brownseds_agn_params.py', **clargs)
sps = model_setup.load_sps(**run_params)
spec_noise, phot_noise = model_setup.load_gp(**run_params)
model = model_setup.load_model(**run_params)
obs = model_setup.load_obs(**run_params)

# set up posterior
def lnprobfn(theta):
    """Given a parameter vector, a dictionary of observational data 
    and a model object, return the ln of the posterior. 
    This requires that an sps object (and if using spectra 
    and gaussian processes, a GP object) be instantiated.
    """

    # Calculate prior probability and exit if not within prior
    lnp_prior = model.prior_product(theta, nested=True)
    if not np.isfinite(lnp_prior):
        return -np.infty
        
    # Generate mean model
    try:
        spec, phot, x = model.mean_model(theta, obs, sps=sps)
    except(ValueError):
        return -np.infty
    vectors = {}  # This would be used for noise model weight functions

    # Calculate likelihoods
    lnp_spec = lnlike_spec(spec, obs=obs, spec_noise=spec_noise)
    lnp_phot = lnlike_phot(phot, obs=obs, phot_noise=phot_noise)

    return lnp_prior + lnp_phot + lnp_spec

# set up prior transform
def prior_transform(u):
    """Simple wrapper for the model prior transform."""
    
    return model.prior_transform(u)

In [ ]:
class Pool(object):
    """A simple wrapper for `dview`."""
    
    def __init__(self, dview):
        self.dview = dview
        
    def map(self, function, tasks):
        return self.dview.map_sync(function, tasks)

# define our pool
pool = Pool(dview)

### Dynamic Nested Sampling

In addition to standard nested sampling, `dynesty` also implements **dynamic nested sampling**, which allows live points to be allocated dynamically. By default, the dynamic nested sampler in dynesty prioritizes posterior inference over evidence (80% posterior vs 20% evidence weighting). It also includes automated stopping criteria to measure posterior and evidence convergence, although by default only the posterior contributions are considered (i.e. 100% posterior vs 0% evidence weighting).

In [ ]:
# initialize sampler
dsampler = dynesty.DynamicNestedSampler(lnprobfn, prior_transform, model.ndim,  # necessary inputs
                                        bound='multi', sample='rwalk',  # how we want to bound/sample
                                        pool=pool, queue_size=nprocs,  # parallel stuff
                                        update_interval=model.ndim*nprocs*1.,  # update interval
                                        **{'walks': 50})  # increase number of walks to be more conservative

In [ ]:
# initial batch
tstart = time.time()  # time it
dsampler.run_nested(dlogz_init=0.01, nlive_init=200, maxbatch=0, use_stop=False)
dresult = dsampler.results
ndur = time.time() - tstart

print('done dynesty (initial, parallel) in {0}s'.format(ndur))

In [ ]:
# subsequent sampling
tstart = time.time()  # time it
dsampler.run_nested(nlive_batch=200, wt_kwargs={'pfrac': 1.0, 'maxfrac': 0.6})
dresult = dsampler.results
ddur = time.time() - tstart

print('done dynesty (dynamic, parallel) in {0}s'.format(ddur))

In [ ]:
# Write the dynesty result object as a pickle  
import pickle
from prospect.io import write_results
outroot = "{0}_{1}".format(run_params['outfile'], int(time.time()))
with open(outroot + '_dns.pkl', 'w') as f:
    pickle.dump(dresult, f)
partext = write_results.paramfile_string(**run_params)

# Write the model as a pickle
write_results.write_model_pickle(outroot + '_model', model, powell=None,
                                 paramfile_text=partext)

In [ ]:
from dynesty import plotting as dyplot

labels = ['logmass', 'z_frac1', 'z_frac2', 'z_frac3', 'z_frac4',
          'z_frac5', 'dust2', 'logzsol', 'dust_index', 'dust1_fraction',
          'duste_qpah', 'duste_gamma', 'duste_umin', 'fagn', 'agn_tau']

In [ ]:
# plot run
rfig, rax = dyplot.runplot(dresult, logplot=True)
tight_layout()

In [ ]:
# plot trace and save figure
fig, axes = subplots(model.ndim, 2, figsize=(20, model.ndim*5))
fig, axes = dyplot.traceplot(dresult, labels=labels, show_titles=True,
                             title_kwargs={'fontsize': 28, 'y': 1.05},
                             fig=(fig, axes))
tight_layout()
fig.savefig(outroot+'_trace.png', bbox_inches='tight')
close()

In [ ]:
# plot corner and save figure
fig, axes = subplots(model.ndim, model.ndim, figsize=(model.ndim*5, model.ndim*5))
fig, axes = dyplot.cornerplot(dresult, smooth=10, color='blue', labels=labels,
                              show_titles=True, title_kwargs={'y': 1.05},
                              fig=(fig, axes))
fig.savefig(outroot+'_corner.png', bbox_inches='tight')
close()

Finally, let's just take a look at a random model drawn from our chain.

In [ ]:
# randomly chosen parameters from chain
idx = np.random.choice(dresult['niter'], p=np.exp(dresult['logwt'] - dresult['logz'][-1]))
theta = dresult['samples'][idx]

# generate model
mspec, mphot, mextra = model.mean_model(theta, obs, sps=sps)

# establish bounds
xmin, xmax = wphot.min() * 0.8, wphot.max() / 0.8
temp = np.interp(np.linspace(xmin, xmax, 10000), wspec * a, mspec)
ymin, ymax = temp.min() * 0.8, temp.max() / 0.8

# set up figure
figure(figsize=(16, 8))

# plot data and model
loglog(wspec * a, mspec, label='Model spectrum',
       lw=0.7, color='navy', alpha=0.7)
errorbar(wphot, mphot, label='Model photometry',
         marker='s', markersize=10, alpha=0.8, ls='', lw=3, 
         markerfacecolor='none', markeredgecolor='blue', 
         markeredgewidth=3)
errorbar(wphot, obs['maggies'], yerr=obs['maggies_unc'], 
         label='Observed photometry', ecolor='red', 
         marker='o', markersize=10, ls='', lw=3, alpha=0.8, 
         markerfacecolor='none', markeredgecolor='red', 
         markeredgewidth=3)

# plot filters
for f in obs['filters']:
    w, t = f.wavelength.copy(), f.transmission.copy()
    while t.max() > 1:
        t /= 10.
    t = 0.1 * (ymax-ymin) * t + ymin
    loglog(w, t, lw=3, color='gray', alpha=0.7)

xlabel('Wavelength [A]')
ylabel('Flux Density [maggies]')
xlim([xmin, xmax])
ylim([ymin, ymax])
legend(loc='best', fontsize=20)
tight_layout()